# Nettoyez et analysez votre jeu de données Nettoyez et analysez votre jeu de données 

## Nettoyer vos données avec python

### Detecter les erreurs

In [20]:
# import des librairies dont nous aurons besoin
import pandas as pd
import numpy as np
import re

In [21]:
# chargement et affichage des données
data = pd.read_csv('personnes.csv')
display(data)

,prenom,email,date_naissance,pays,taille
0,Leila,leila@example.com,23/01/1990,France,1.49m
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m
2,Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,153cm
3,Marc,"marco23@example.com, mc23@supermail.eu",10/02/1978,France,1.65m
4,Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.34m
5,Hanna,hanna2019@supermail.eu,01/01/1970,24,3.45m
6,samuël,samuel_329@example.com,NaN,Bénin,1.45m


#### Valeurs manquantes

In [22]:
#Detecter les valeurs manquante (les NULL)
print(data.isnull().sum())

prenom            0
email             0
date_naissance    1
pays              1
taille            0
dtype: int64


#### Doublons

In [23]:
#Detecter les doublons
data.loc[data['email'].duplicated(keep=False),:]

,prenom,email,date_naissance,pays,taille
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m
6,samuël,samuel_329@example.com,NaN,Bénin,1.45m


### Traitez les erreurs : 

#### Theorie

`# Nouvelle dataframe :`

`data['nom_colonne'] = nouvelle_colonne`

`mask = # condition à vérifier pour cibler spécifiquement certaines lignes`

`data.loc[mask, 'ma_colonne'] = nouvelles_valeurs`

#### Traitez les pays :

In [24]:
display(data['pays'])

0           France
1              NaN
2    Côte d'ivoire
3           France
4       Madagascar
5               24
6            Bénin
Name: pays, dtype: object

- Le probleme etait que il y a la valeur 24 qui ne correspond pas a un pays
- Pour regler cette erreurs nous allons creer un dictionnaire ou nous allons mettre nos pays
- Nous alons donc verifier si le pays correspond ou non a un pays du dictionnaire
- Si il correspond alors tout est bon
- Si ça correspond pas alors on met un ***NaN***

In [25]:
VALID_COUNTRIES = ['France', 'Côte d\'ivoire', 'Madagascar', 'Bénin', 'Allemagne'
                  , 'USA']
mask = ~data['pays'].isin(VALID_COUNTRIES)
data.loc[mask, 'pays'] = np.NaN
display(data['pays'])

0           France
1              NaN
2    Côte d'ivoire
3           France
4       Madagascar
5              NaN
6            Bénin
Name: pays, dtype: object

On remarque que la valeur ***24*** qui n'etait pas dans le dictionnaire er devnu la valeur ***Nan***

#### Traitez les e-mails

In [26]:
data['email'] 

0                         leila@example.com
1                    samuel_329@example.com
2                  choupipoune@supermail.eu
3    marco23@example.com, mc23@supermail.eu
4                   helloworld@supermail.eu
5                    hanna2019@supermail.eu
6                    samuel_329@example.com
Name: email, dtype: object

Au tour des e-mails, maintenant ! Le problème avec cette colonne, c'est qu'il y a parfois 2 adresses e-mail par ligne. Nous ne souhaitons prendre que la première

In [27]:
data['email'] = data['email'].str.split(',', n=1, expand=True)[0]
display(data['email'])

0           leila@example.com
1      samuel_329@example.com
2    choupipoune@supermail.eu
3         marco23@example.com
4     helloworld@supermail.eu
5      hanna2019@supermail.eu
6      samuel_329@example.com
Name: email, dtype: object

Nous avons plus que un email par ligne

#### Traitez les tailles

In [28]:
display(data['taille'])

0    1.49m
1    1.67m
2    153cm
3    1.65m
4    1.34m
5    3.45m
6    1.45m
Name: taille, dtype: object

On remarque une taille impossible et un taille en cm

Ici nous decidons de garder seulement les taille en m. Et puis de donner les taille manquantes grace a la moyenne des autre tailles. Nous ne faisons pax la conversion cm -> m

In [29]:
data['taille'] = data['taille'].str[:-1]
data['taille'] = pd.to_numeric(data['taille'], errors='coerce')
display(data['taille'])

0    1.49
1    1.67
2     NaN
3    1.65
4    1.34
5    3.45
6    1.45
Name: taille, dtype: float64

In [30]:
data.loc[data['taille'].isnull(), 'taille'] = data['taille'].mean()
data['taille']=data['taille'].round(2)
display(data['taille'])

0    1.49
1    1.67
2    1.84
3    1.65
4    1.34
5    3.45
6    1.45
Name: taille, dtype: float64

On remarque que il rest la taille de 3.45 m 
On va essayrer de la traiter

In [31]:
data.loc[(data['taille']<=1.0) | (data['taille']>= 2.5),'taille'] = data['taille'].mean()
data['taille']=data['taille'].round(2)
display(data['taille'])

0    1.49
1    1.67
2    1.84
3    1.65
4    1.34
5    1.84
6    1.45
Name: taille, dtype: float64

On remarque que nous avons remplacer la valeur trop haute

#### Triez les dates

Nous allons spécifier un certain format de date, et transformer notre variable dans un type adéquat. Les formats non adequat vont etre transformer en NaN grace à ***coerce*** comme pour les tailles

In [32]:
data['date_naissance'] = pd.to_datetime(data['date_naissance'], format='%d/%m/%Y', errors='coerce')
display(data['date_naissance'])

0   1990-01-23
1   2001-09-20
2          NaT
3   1978-02-10
4   2008-03-05
5   1970-01-01
6          NaT
Name: date_naissance, dtype: datetime64[ns]

### Exercice chapitre 2

In [33]:
#importation des données
df_operation = pd.read_csv('operations.csv')
df_operation.head()

,date_operation,libelle,montant,solde_avt_ope,categ
0,2023-03-31,DON XX XX XX XX XX XX XX,-1.44,1515.25,AUTRE
1,2023-04-03,CARTE XX XX RAPT XX,-24.00,1513.81,TRANSPORT
2,2023-04-03,CARTE XX XX RAPT XX,-73.00,1489.81,TRANSPORT
3,2023-04-03,VIREMENT XX XX XX XX XX XX XX XX XX XX XX XX,676.00,1416.81,AUTRE
4,2023-04-03,VIREMENT XX XX XX XX XX XX,4.80,2092.81,AUTRE


<p>Instruction</p>
<blockquote> Plusieurs erreurs se sont glissées dans ce jeu de données. Votre mission, si toutefois vous l'acceptez, va être de les trouver et de proposer des solutions adéquates pour les gérer. Vous aurez besoin dans tous les cas de nettoyer votre fichier avant de passer à la suite !</blockquote>

In [40]:
df_operation=df_operation.loc[df_operation['libelle'].duplicated(keep=False),:]

**Premier étape : traitez les dates opérations**
- verifier le format
- Si pas le bon format on le rend `null`

In [34]:
pd.to_datetime(df_operation['date_operation'], errors='coerce')

0     2023-03-31
1     2023-04-03
2     2023-04-03
3     2023-04-03
4     2023-04-03
         ...    
304   2023-10-05
305   2023-10-05
306   2023-10-06
307   2023-10-06
308   2023-10-06
Name: date_operation, Length: 309, dtype: datetime64[ns]

**2 : Gerer les libellées**

Je ne voit pas d'erreur dans les libellé et je ne vois pas comment les gerer.


**3 : Verifier que le montant avant opertion et le solde concorde**

Il faut verifier $ \forall i \in [\![1:n]\!], \ montant[i]=solde\_avt\_ope[i+1]-solde\_avt\_ope[i]$

In [35]:
i=0

display(df_operation.loc[:,"montant"].loc[i])
operation_tmp = df_operation["solde_avt_ope"].loc[i+1] - df_operation["solde_avt_ope"].loc[i]
operation_tmp= round(operation_tmp,2)
display(operation_tmp)

display(df_operation["solde_avt_ope"][i])

display(df_operation["montant"][i] + df_operation["solde_avt_ope"][i])

-1.44

-1.44

1515.25

1513.81

In [36]:
array_operation = df_operation.to_numpy()
array_operation[1,3]

1513.81

In [38]:
array_operation = df_operation.to_numpy()
#montant : [i,2] / solde_avt_ope : [i,3]
for i in range(0,array_operation.shape[0]-1):
    print( round(array_operation[i+1,3] - array_operation[i,3],2))
    print(array_operation[i,2])
    if (array_operation[i,2] != round(array_operation[i+1,3] - array_operation[i,3],2)):
        array_operation[i+1,3] = round(array_operation[i,2] + array_operation[i,3],2)

-1.44
-1.44
-24.0
-24.0
-73.0
-73.0
676.0
676.0
4.8
4.8
-14.39
-14.39
-15.2
-15.2
-12.0
-12.0
-7.02
-7.02
-6.8
-6.8
-1.84
-1.84
992.8
992.8
-24.0
-24.0
1.6
1.6
-84.4
-84.4
-12.42
-12.42
-6.0
-6.0
-20.64
-20.64
-1.6
-1.6
-42.42
-42.42
-2.7
-2.7
836.0
836.0
-7.56
-7.56
869.44
869.44
-48.0
-48.0
-28.96
-28.96
-280.0
-280.0
-1.6
-1.6
-8.8
-8.8
-0.8
-0.8
-345.0
-345.0
1.6
1.6
-10.32
-10.32
-8.72
-8.72
-24.0
-24.0
-74.58
-74.58
-1.7
-1.7
-13.6
-13.6
-7.2
-7.2
-10.43
-10.43
-7.84
-7.84
-11.64
-11.64
-9.2
-9.2
0.0
-32.67
0.0
-32.67
22.67
-10.0
30.77
-1.9
23.56
-9.11
31.27
-1.4
-3.33
-36.0
20.43
-12.24
79.87
47.2
24.19
-8.48
15.97
-16.7
22.27
-10.4
0.67
-32.0
-40.33
-73.0
18.28
-14.39
15.51
-17.16
708.67
676.0
29.19
-3.48
30.83
-1.84
25.65
-7.02
31.07
-1.6
31.55
-1.12
31.23
-1.44
0.67
-32.0
23.07
-9.6
-9.78
-42.45
31.07
-1.6
27.07
-5.6
-312.33
-345.0
-0.88
-33.55
21.67
-11.0
19.71
-12.96
24.67
-8.0
31.07
-1.6
20.67
-12.0
21.27
-11.4
0.67
-32.0
30.27
-2.4
22.95
-9.72
968.99
936.32
-105.57
-138.2